In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Reload all modules imported with %aimport
%load_ext autoreload
%autoreload 1

In [2]:
import helper
%aimport helper

helper = helper.Helper()

In [3]:
import tensorflow as tf
import numpy as np
import os

## Load the data

In [4]:
data_dir = np.load('./data/train_test_data.npz')
X_train = data_dir['X_train']
X_test = data_dir['X_test']
y_train = data_dir['y_train']
y_test = data_dir['y_test']
data = np.concatenate((X_train, X_test))
labels = np.concatenate((y_train, y_test))

## Load models and histories

In [5]:
model_name0 = "CNN + Head"
model0 = helper.loadModel(model_name0)
score0 = model0.evaluate(X_test, y_test, verbose=0)
history0 = helper.loadHistory(model_name0)

model_name1 = "4CNNs + Head"
model1 = helper.loadModel(model_name1)
score1 = model1.evaluate(X_test, y_test, verbose=0)
history1 = helper.loadHistory(model_name1)

### 1. Model structure test

In [6]:
### BEGIN HIDDEN TESTS
# Set Classifier activation and loss depending on shape of target
if helper.y_OHE(y_train):
    num_cases = 2
    activation = "softmax"
    loss = 'categorical_crossentropy'
else:
    num_cases = 1
    activation = "sigmoid"
    loss = 'binary_crossentropy'
  

## Model0 Head Only

n_samples, width, height, channel = data.shape
CNN_layer = model0.get_layer(name='CNN_1')
dense_layer = model0.get_layer(name='dense_head')
CNN_features = 32
# Convolutional layer tests
assert CNN_layer.input.shape[1:] == tf.TensorShape(data.shape[1:])
assert CNN_layer.output.shape[1:] == tf.TensorShape([width, height, CNN_features])

# Head layer
assert dense_layer.output.shape[1:] == tf.TensorShape([num_cases])
### END HIDDEN TESTS

In [7]:
### BEGIN HIDDEN TESTS
CNN_features_1 = 32
CNN_features_2 = 32
CNN_features_3 = 64
CNN_features_4 = 64
MaxPool_size = 2

CNN_1 = model1.get_layer(name='CNN_1')
CNN_2 = model1.get_layer(name='CNN_2')
CNN_3 = model1.get_layer(name='CNN_3')
CNN_4 = model1.get_layer(name='CNN_4')
dense_head = model1.get_layer(name='dense_head')

# CNN 1
assert CNN_1.input.shape[1:] == tf.TensorShape(data.shape[1:])
assert CNN_1.output.shape[1:] == tf.TensorShape([width, height, CNN_features_1])

# CNN 2
assert CNN_2.input.shape[1:] == tf.TensorShape([width, height, CNN_features_1])
assert CNN_2.output.shape[1:] == tf.TensorShape([width, height, CNN_features_2])

# CNN 3
assert CNN_3.input.shape[1:] == tf.TensorShape([int(width/MaxPool_size), int(height/MaxPool_size), CNN_features_2])
assert CNN_3.output.shape[1:] == tf.TensorShape([int(width/MaxPool_size), int(height/MaxPool_size), CNN_features_3])

# CNN 4
assert CNN_4.input.shape[1:] == tf.TensorShape([int(width/MaxPool_size), int(height/MaxPool_size), CNN_features_3])
assert CNN_4.output.shape[1:] == tf.TensorShape([int(width/MaxPool_size), int(height/MaxPool_size), CNN_features_4])

# Dense head
assert dense_head.output.shape[1:] == tf.TensorShape([num_cases])

### END HIDDEN TESTS

### 2. Training result test

In [8]:
### BEGIN HIDDEN TESTS

# Create the key with which to look up accuracy, val_accuracy.  It depends on how the model was compiled.
acc_string = helper.acc_key(model=model1)
acc_metric, val_acc_metric = acc_string, "val_" + acc_string

assert history0[acc_string][-1] > 0.85
assert history0[val_acc_metric][-1] > 0.85
### END HIDDEN TESTS

In [9]:
### BEGIN HIDDEN TESTS
assert history1[acc_string][-1] > history0[acc_string][-1]
assert history1[val_acc_metric][-1] > history0[val_acc_metric][-1]
### END HIDDEN TESTS

### 3. Evaluate the model

In [10]:
### BEGIN HIDDEN TESTS
assert score0[1] > 0.85
### END HIDDEN TESTS

In [11]:
### BEGIN HIDDEN TESTS
assert score1[1] > score0[1]
### END HIDDEN TESTS

AssertionError: 